In [ ]:
# Clone github repository
!git clone --branch model_editing https://github.com/AlessandroMaini/federated-learning-project.git

In [ ]:
%cd federated-learning-project

In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from data.cifar100_loader import get_cifar100_loaders
from models.prepare_model import get_dino_vits16_model, freeze_backbone, unfreeze_backbone, freeze_head, unfreeze_head
from models.hyperparameter_tuning import run_grid_search
from eval import evaluate
from train import train
from models.model_editing import mask_calculator
import matplotlib.pyplot as plt

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Directories
CHECKPOINT_DIR = './checkpoints'
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

In [ ]:
criterion = nn.CrossEntropyLoss()

In [ ]:
model = get_dino_vits16_model(device)
# Freeze backbone
freeze_backbone(model)

In [ ]:
# Train on full training set (train + val)
full_train_loader, _, test_loader = get_cifar100_loaders(val_split=0.0)

In [ ]:
# Perform brief pre-training on the full training set
start_epoch = 0
warmup_epochs = 3
best_test_acc = 0.0

warmup_train_loss = []
warmup_train_acc = []
warmup_test_loss = []
warmup_test_acc = []

In [ ]:
best_cfg = {"lr": 0.005, "momentum": 0.9}

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=best_cfg['lr'], momentum=best_cfg['momentum'], weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=warmup_epochs)

In [ ]:
for epoch in range(start_epoch, start_epoch + warmup_epochs):
    train_loss, train_acc = train(model, full_train_loader, optimizer, criterion, device)
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    scheduler.step()
    
    warmup_train_loss.append(train_loss)
    warmup_train_acc.append(train_acc)
    warmup_test_loss.append(test_loss)
    warmup_test_acc.append(test_acc)

    print(f"Epoch {epoch+1}/{start_epoch + warmup_epochs}")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"  Test Loss:  {test_loss:.4f} | Test Acc:  {test_acc:.4f}")

torch.save(model.state_dict(), os.path.join(CHECKPOINT_DIR, f'pre_trained_model_centralized.pth'))

In [ ]:
# Plot the training and test loss
plt.plot(warmup_train_loss, label='Train Loss')
plt.plot(warmup_test_loss, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Plot the training and test accuracy
plt.plot(warmup_train_acc, label='Train Accuracy')
plt.plot(warmup_test_acc, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Load the pre-trained model
model.load_state_dict(torch.load(os.path.join(CHECKPOINT_DIR, 'pre_trained_model_centralized.pth')))
# Unfreeze the backbone
unfreeze_backbone(model)
# Freeze the head
freeze_head(model)

# Compute the mask
mask = mask_calculator(model, full_train_loader.dataset, device)

In [ ]:
# Train the model with the mask
start_epoch = 0
num_epochs = 10
best_test_acc = 0.0

hist_train_loss = []
hist_train_acc = []
hist_test_loss = []
hist_test_acc = []

In [ ]:
optimizer = optim.SGD(model.parameters(), lr=best_cfg['lr'], momentum=best_cfg['momentum'], weight_decay=5e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs)

In [ ]:
## If you want to load a checkpoint, uncomment the following lines
# PATH_TO_CHECKPOINT = "./checkpoints/dino_vits16_epoch30.pth"
# checkpoint = torch.load(PATH_TO_CHECKPOINT, map_location=device)

# model.load_state_dict(checkpoint['model_state_dict'])
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
# start_epoch = checkpoint['epoch']

In [ ]:
for epoch in range(start_epoch, start_epoch + num_epochs):
    train_loss, train_acc = train(model, full_train_loader, optimizer, criterion, device, grad_mask=mask)
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    scheduler.step()
    
    hist_train_loss.append(train_loss)
    hist_train_acc.append(train_acc)
    hist_test_loss.append(test_loss)
    hist_test_acc.append(test_acc)

    print(f"Epoch {epoch+1}/{start_epoch + num_epochs}")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"  Test Loss:  {test_loss:.4f} | Test Acc:  {test_acc:.4f}")

    if test_acc > best_test_acc:
        best_test_acc = test_acc
        torch.save({
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict()
        }, os.path.join(CHECKPOINT_DIR, 'best_model_overall.pth'))

    if (epoch + 1) % 10 == 0:
        checkpoint = {
            'epoch': epoch + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict()
        }
        torch.save(checkpoint, os.path.join(CHECKPOINT_DIR, f'centralized_edited_epoch{epoch+1}.pth'))

In [ ]:
# Plot the training and test loss
plt.plot(hist_train_loss, label='Train Loss')
plt.plot(hist_test_loss, label='Test Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Plot the training and test accuracy
plt.plot(hist_train_acc, label='Train Accuracy')
plt.plot(hist_test_acc, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()